In [8]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langgraph.checkpoint.memory import InMemorySaver

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [10]:
class jokeState(TypedDict):
    topic : str
    joke : str
    explaination : str

In [15]:
def create_joke(state : jokeState):
    topic = state["topic"]
    prompt = PromptTemplate.from_template("Generate a funny joke on the topic.\n\n topic = {topic}")
    response = llm.invoke(prompt.format_prompt(topic=topic)).content
    return {"joke" : response}

In [16]:
def explain_joke(state : jokeState):
    topic = state["topic"]
    joke = state["joke"]
    prompt = PromptTemplate.from_template("Explain the generated joke on the topic.\n\n topic = {topic} \n joke = {joke}")
    response = llm.invoke(prompt.format_prompt(topic=topic, joke=joke)).content
    return {"explaination" : response}


In [17]:
graph = StateGraph(jokeState)
graph.add_node("Joke", create_joke)
graph.add_node("Explain", explain_joke)
graph.add_edge(START, "Joke")
graph.add_edge("Joke", "Explain")
graph.add_edge("Explain", END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [18]:
config1 = {"configurable" : {"thread_id": "1"}}
workflow.invoke({"topic":"pizza"}, config=config1)

{'topic': 'pizza',
 'joke': 'What do you call a sleeping pizza?\nA **pizzzzza**!',
 'explaination': 'This is a classic phonetic pun! Here\'s the breakdown:\n\n1.  **The Setup:** "What do you call a sleeping pizza?" This immediately tells you the answer will involve both "pizza" and something related to "sleep."\n\n2.  **The Punchline:** "A **pizzzzza**!"\n\n3.  **The Explanation:**\n    *   The base word is "pizza."\n    *   The "zzzz" sound is the universal onomatopoeia (sound word) for **snoring** or **sleeping** (like you\'d see in a cartoon bubble above a sleeping character).\n    *   The joke takes the word "pizza" and stretches out the "z" sound, incorporating the "zzzz" sleep sound directly into the word itself. It makes "pizza" *sound* like it\'s sleeping or snoring.\n\nIt\'s a simple, lighthearted pun that plays on the sound of the word "pizza" and the common representation of sleep.'}

In [20]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'What do you call a sleeping pizza?\nA **pizzzzza**!', 'explaination': 'This is a classic phonetic pun! Here\'s the breakdown:\n\n1.  **The Setup:** "What do you call a sleeping pizza?" This immediately tells you the answer will involve both "pizza" and something related to "sleep."\n\n2.  **The Punchline:** "A **pizzzzza**!"\n\n3.  **The Explanation:**\n    *   The base word is "pizza."\n    *   The "zzzz" sound is the universal onomatopoeia (sound word) for **snoring** or **sleeping** (like you\'d see in a cartoon bubble above a sleeping character).\n    *   The joke takes the word "pizza" and stretches out the "z" sound, incorporating the "zzzz" sleep sound directly into the word itself. It makes "pizza" *sound* like it\'s sleeping or snoring.\n\nIt\'s a simple, lighthearted pun that plays on the sound of the word "pizza" and the common representation of sleep.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '',

In [22]:
# for intermediate values
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'What do you call a sleeping pizza?\nA **pizzzzza**!', 'explaination': 'This is a classic phonetic pun! Here\'s the breakdown:\n\n1.  **The Setup:** "What do you call a sleeping pizza?" This immediately tells you the answer will involve both "pizza" and something related to "sleep."\n\n2.  **The Punchline:** "A **pizzzzza**!"\n\n3.  **The Explanation:**\n    *   The base word is "pizza."\n    *   The "zzzz" sound is the universal onomatopoeia (sound word) for **snoring** or **sleeping** (like you\'d see in a cartoon bubble above a sleeping character).\n    *   The joke takes the word "pizza" and stretches out the "z" sound, incorporating the "zzzz" sleep sound directly into the word itself. It makes "pizza" *sound* like it\'s sleeping or snoring.\n\nIt\'s a simple, lighthearted pun that plays on the sound of the word "pizza" and the common representation of sleep.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': ''

In [23]:
config2 = {"configurable" : {"thread_id": "2"}}
workflow.invoke({"topic":"Potato"}, config=config2)

{'topic': 'Potato',
 'joke': 'What do you call a lazy potato?\nA couch potato!',
 'explaination': 'This joke is a classic **pun** that plays on the well-known English idiom "couch potato."\n\nHere\'s the breakdown:\n\n1.  **The Topic:** Potato (the vegetable).\n2.  **The Setup:** "What do you call a lazy potato?" This sets up an expectation for a name that describes a *vegetable* being lazy.\n3.  **The Punchline:** "A couch potato!"\n4.  **The Pun/Idiom:**\n    *   A "couch potato" is an informal term for a person who spends a great deal of time sitting or lying on a couch, typically watching television or playing video games, and generally being inactive or lazy.\n    *   The joke takes this figurative meaning (a lazy *person*) and applies it literally to a *potato* (the vegetable).\n\nThe humor comes from the unexpected literal interpretation. We know what a "couch potato" refers to in everyday language, so applying it directly to an actual potato creates a funny, whimsical image of 

In [24]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'Potato', 'joke': 'What do you call a lazy potato?\nA couch potato!', 'explaination': 'This joke is a classic **pun** that plays on the well-known English idiom "couch potato."\n\nHere\'s the breakdown:\n\n1.  **The Topic:** Potato (the vegetable).\n2.  **The Setup:** "What do you call a lazy potato?" This sets up an expectation for a name that describes a *vegetable* being lazy.\n3.  **The Punchline:** "A couch potato!"\n4.  **The Pun/Idiom:**\n    *   A "couch potato" is an informal term for a person who spends a great deal of time sitting or lying on a couch, typically watching television or playing video games, and generally being inactive or lazy.\n    *   The joke takes this figurative meaning (a lazy *person*) and applies it literally to a *potato* (the vegetable).\n\nThe humor comes from the unexpected literal interpretation. We know what a "couch potato" refers to in everyday language, so applying it directly to an actual potato creates a funny, 

In [25]:
# for intermediate values
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Potato', 'joke': 'What do you call a lazy potato?\nA couch potato!', 'explaination': 'This joke is a classic **pun** that plays on the well-known English idiom "couch potato."\n\nHere\'s the breakdown:\n\n1.  **The Topic:** Potato (the vegetable).\n2.  **The Setup:** "What do you call a lazy potato?" This sets up an expectation for a name that describes a *vegetable* being lazy.\n3.  **The Punchline:** "A couch potato!"\n4.  **The Pun/Idiom:**\n    *   A "couch potato" is an informal term for a person who spends a great deal of time sitting or lying on a couch, typically watching television or playing video games, and generally being inactive or lazy.\n    *   The joke takes this figurative meaning (a lazy *person*) and applies it literally to a *potato* (the vegetable).\n\nThe humor comes from the unexpected literal interpretation. We know what a "couch potato" refers to in everyday language, so applying it directly to an actual potato creates a funny,